In [1]:
from pkg.utils.utils_python3 import *

CONVERTED_PATH = "./data/converted"
SCENE_FILENAME = "scene.pkl"
JOINT_NUM = 13
gtimer = GlobalTimer()
# ## Load Global params
DATASET_LIST = sorted(filter(lambda x: x not in ["backup", "converted"], os.listdir(CONVERTED_PATH)))
DATASET = DATASET_LIST[0]
CURRENT_PATH = os.path.join(CONVERTED_PATH, DATASET)
#Iterate world
WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
gtimer.reset()
SCENE_TUPLE_LIST = []
for WORLD in WORLD_LIST:
    WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
    # Iterate scene
    SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
    for SCENE in SCENE_LIST:
        SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
        ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
        for ACTION in ACTION_LIST:
            N_action = get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION)
            for i_act in range(N_action):
                SCENE_TUPLE_LIST.append((CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM))
                gtimer.tic("load_scene_data")
                scene_data, success, skey = load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM)
                gtimer.toc("load_scene_data")
print(gtimer)

TRAIN_RATIO = 0.7
import random
N_train  = int(len(SCENE_TUPLE_LIST)*TRAIN_RATIO)
N_test = len(SCENE_TUPLE_LIST) - N_train
train_set = SCENE_TUPLE_LIST[:N_train]
test_set = SCENE_TUPLE_LIST[N_train:]

load_scene_data: 	5754.0 ms/6276 = 0.917 ms (0.637/3.808)



In [78]:
import tensorflow as tf

from tensorflow.keras import layers as KL
from tensorflow.keras import Model

class DenseBN:
    def __init__(self, units, name, use_bias=True, activation="relu"):
        self.dense = KL.Dense(units, name='dense_' + name, use_bias=use_bias)
        self.bn = KL.BatchNormalization(name='bn_' + name)
        self.ac = KL.Activation(activation)
        
    def __call__(self, x, training=False):
        x = self.dense(x)
        x = self.bn(x, training=training)
        x = self.ac(x)
        return x
        

class IdentityBlock:
    def __init__(self, kernel_size, filters, stage, block,
                 use_bias=True, ConvLayer=KL.Conv2D):
        nb_filter1, nb_filter2, nb_filter3 = filters
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        
        self.conv1 = ConvLayer(nb_filter1, 1, name=conv_name_base + '2a',
                               use_bias=use_bias)
        self.bn1 = KL.BatchNormalization(name=bn_name_base + '2a')
        self.ac1 = KL.Activation('relu')
        
        self.conv2 = ConvLayer(nb_filter2, kernel_size, padding="same",
                               name=conv_name_base + '2b', use_bias=use_bias)
        self.bn2 = KL.BatchNormalization(name=bn_name_base + '2b')
        self.ac2 = KL.Activation('relu')
        
        self.conv3 = ConvLayer(nb_filter3, 1, name=conv_name_base + '2c',
                               use_bias=use_bias)
        self.bn3 = KL.BatchNormalization(name=bn_name_base + '2c')
        self.add = KL.Add()
        self.ac3 = KL.Activation('relu', name='res' + str(stage) + block + '_out')
        
    def __call__(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.ac1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.ac2(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)

        x = self.add([x, input_tensor])
        x = self.ac3(x)
        return x
    
class IdentityBlock3D(IdentityBlock):
    def __init__(self, *args, **kwargs):
        super(IdentityBlock3D, self).__init__(*args, ConvLayer=KL.Conv3D, **kwargs)
    
    

class ConvBlock:
    def __init__(self, kernel_size, filters, stage, block, strides=2,
                 use_bias=True, ConvLayer=KL.Conv2D):
        nb_filter1, nb_filter2, nb_filter3 = filters
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        
        self.conv1 = ConvLayer(nb_filter1, 1, strides=strides, 
                               name=conv_name_base + '2a', use_bias=use_bias)
        self.bn1 = KL.BatchNormalization(name=bn_name_base + '2a')
        self.ac1 = KL.Activation('relu')
        
        self.conv2 = ConvLayer(nb_filter2, kernel_size, padding="same",
                               name=conv_name_base + '2b', use_bias=use_bias)
        self.bn2 = KL.BatchNormalization(name=bn_name_base + '2b')
        self.ac2 = KL.Activation('relu')
        
        self.conv3 = ConvLayer(nb_filter3, 1, name=conv_name_base + '2c',
                               use_bias=use_bias)
        self.bn3 = KL.BatchNormalization(name=bn_name_base + '2c')

        self.convs = ConvLayer(nb_filter3, 1, strides=strides,
                               name=conv_name_base + '1', use_bias=use_bias)
        self.bns = KL.BatchNormalization(name=bn_name_base + '1')
    
        self.add = KL.Add()
        self.ac3 = KL.Activation('relu', name='res' + str(stage) + block + '_out')
        
    def __call__(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.ac1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.ac2(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)
        
        shortcut = self.convs(input_tensor)
        shortcut = self.bns(shortcut, training=training)

        x = self.add([x, shortcut])
        x = self.ac3(x)
        return x
    
class ConvBlock3D(ConvBlock):
    def __init__(self, *args, **kwargs):
        super(ConvBlock3D, self).__init__(*args, ConvLayer=KL.Conv3D, **kwargs)
    

class ResNet:
    def __init__(self, architecture="resnet50", stage1=None, stage2=[64, 64, 256], stage3=[128, 128, 512], 
                 stage4=[256, 256, 1024], stage5=None, 
                 ConvLayer=KL.Conv2D, ZeroPadding=KL.ZeroPadding2D, MaxPool=KL.MaxPooling2D, 
                 ConvBlock=ConvBlock, IdentityBlock=IdentityBlock):
        """Build a ResNet graph.
            architecture: Can be resnet50 or resnet101
            stage5: recommanded default values = [512, 512, 2048]. If None, stage5 of the network is not created
        """
        assert architecture in ["resnet50", "resnet101"]
        self.stage1 = stage1 is not None
        self.stage5 = stage5 is not None
        
        if self.stage1:
            # Stage 1  output size = 1/1
            self.cb1a = ConvBlock(3, stage1, stage=1, block='a', strides=1)
            self.ib1b = IdentityBlock(3, stage2, stage=1, block='b')
            self.ib1c = IdentityBlock(3, stage2, stage=1, block='c')
            stride_2 = 2
        else:
            # Stage 1 original output size = 1/4
            self.cv1 = ConvLayer(64, 7, strides=2, 
                                 name='conv1', padding="same", use_bias=True)
            self.bn1 = KL.BatchNormalization(name='bn_conv1')
            self.ac1 = KL.Activation('relu')
            self.mp1 = MaxPool(3, strides=2, padding="same")
            stride_2 = 1
        
        # Stage 2  output size = 1/stride_2
        self.cb2a = ConvBlock(3, stage2, stage=2, block='a', strides=stride_2)
        self.ib2b = IdentityBlock(3, stage2, stage=2, block='b')
        self.ib2c = IdentityBlock(3, stage2, stage=2, block='c')
        
        # Stage 3  output size = 1/2
        self.cb3a = ConvBlock(3, stage3, stage=3, block='a')
        self.ib3b = IdentityBlock(3, stage3, stage=3, block='b')
        self.ib3c = IdentityBlock(3, stage3, stage=3, block='c')
        self.ib3d = IdentityBlock(3, stage3, stage=3, block='d')
        
        # Stage 4  output size = 1/2
        self.cb4a = ConvBlock(3, stage4, stage=4, block='a')
        block_count = {"resnet50": 5, "resnet101": 22}[architecture]
        self.ib4_list = []
        for i in range(block_count):
            self.ib4_list.append(
                IdentityBlock(3, stage4, stage=4, block=chr(98 + i))
            )
            
        # Stage 5  output size = 1/2
        if self.stage5:
            self.cb5a = ConvBlock(3, stage5, stage=5, block='a')
            self.ib5b = IdentityBlock(3, stage5, stage=5, block='b')
            self.ib5c = IdentityBlock(3, stage5, stage=5, block='c')

    def __call__(self, input_image, training=False):
        x = input_image
        # Stage 1
        if self.stage1:
            x = self.cb1a(x, training=training)
            x = self.ib1b(x, training=training)
            C1 = x = self.ib1c(x, training=training)
        else:
            x = self.cv1(x)
            x = self.bn1(x, training=training)
            x = self.ac1(x)
            C1 = x = self.mp1(x)
        
        # Stage 2
        x = self.cb2a(x, training=training)
        x = self.ib2b(x, training=training)
        C2 = x = self.ib2c(x, training=training)
        
        # Stage 3
        x = self.cb3a(x, training=training)
        x = self.ib3b(x, training=training)
        x = self.ib3c(x, training=training)
        C3 = x = self.ib3d(x, training=training)
        
        # Stage 4
        x = self.cb4a(x, training=training)
        for ib4 in self.ib4_list:
            x = ib4(x, training=training)
        C4 = x
            
        # Stage 5
        if self.stage5:
            x = self.cb5a(x, training=training)
            x = self.ib5b(x, training=training)
            C5 = x = self.ib5c(x, training=training)
            return [C1, C2, C3, C4, C5]
        else:
            return [C1, C2, C3, C4]
            
    
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [86]:
class ResNetModelTP(Model):
    def __init__(self):
        super(ResNetModelTP, self).__init__()
        self.resnet = ResNet(architecture="resnet50", stage1=[256, 256, 512], 
                             stage2=[256, 256, 512], stage3=[512, 512, 1024], 
                             stage4=[512, 512, 1024], stage5=[512, 512, 1024], 
                             ConvLayer=KL.Conv3D, ZeroPadding=KL.ZeroPadding3D, MaxPool=KL.MaxPooling3D, 
                             ConvBlock=ConvBlock3D, IdentityBlock=IdentityBlock3D)
        self.flatten = KL.Flatten()
        self.gp_list = []
        for _ in range(4):
            self.gp_list.append(KL.GlobalMaxPool3D())
        self.add1024 = KL.Add()
        self.add512 = KL.Add()
        self.dens1 = DenseBN(512, "dens2")
        self.dens2 = DenseBN(256, "dens3")
        self.dens3 = Dense(2)

    def call(self, x, training=False):
        features = self.resnet(x, training=training)
        features_pool = []
        for gp, ft in zip(self.gp_list features[:-1]):
            features_pool.append(gp(ft))
        x = self.flatten(features[-1])
        x = self.add1024(features_pool[2:]+[x])
        x = self.dens1(x, training=training)
        x = self.add512(features_pool[:2]+[x])
        x = self.dens2(x, training=training)
        return self.dens3(x)

# Create an instance of the model
model = ResNetModelTP()

In [87]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [88]:
EPOCHS = 5
BATCH_SIZE = 16
LOG_STEP = 100

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(train_set)
    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in train_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            train_step(np.array(scene_batch, dtype=np.float32), np.array(success_batch, dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')

    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in test_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            test_step(np.array(scene_batch, dtype=np.float32), np.array(success_batch,dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.7347690463066101, Accuracy: 59.03284454345703, Test Loss: 0.665594756603241, Test Accuracy: 65.43803405761719
Epoch 2, Loss: 0.6926104426383972, Accuracy: 60.561134338378906, Test Loss: 0.6775022745132446, Test Accuracy: 62.5
Epoch 3, Loss: 0.6766912937164307, Accuracy: 62.614051818847656, Test Loss: 0.6804439425468445, Test Accuracy: 62.12607192993164
Epoch 4, Loss: 0.6693775653839111, Accuracy: 62.18065643310547, Test Loss: 0.7040701508522034, Test Accuracy: 54.00640869140625
Epoch 5, Loss: 0.6631377935409546, Accuracy: 62.750911712646484, Test Loss: 0.6714820265769958, Test Accuracy: 60.95085906982422


In [ ]:
tf.__version__

In [11]:
np.array(success_batch,dtype=np.int)

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0])

In [12]:
predictions = model(np.array(scene_batch, dtype=np.float32), training=False)

In [13]:
dtest = Dense(1, activation='sigmoid')

In [14]:
dtest((tf.constant([[10]])))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.99995804]], dtype=float32)>

In [14]:
predictions

<tf.Tensor: shape=(27, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)>

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.14230504631996155, Accuracy: 95.68333435058594, Test Loss: 0.05939216911792755, Test Accuracy: 97.91999816894531
Epoch 2, Loss: 0.04611682891845703, Accuracy: 98.52166748046875, Test Loss: 0.050787363201379776, Test Accuracy: 98.18999481201172
Epoch 3, Loss: 0.024921394884586334, Accuracy: 99.20832824707031, Test Loss: 0.0528109110891819, Test Accuracy: 98.23999786376953
Epoch 4, Loss: 0.016222544014453888, Accuracy: 99.46666717529297, Test Loss: 0.05345192924141884, Test Accuracy: 98.5
Epoch 5, Loss: 0.010034429840743542, Accuracy: 99.66666412353516, Test Loss: 0.05353892967104912, Test Accuracy: 98.41999816894531


In [10]:
print(images.shape, labels.shape)

(32, 28, 28, 1) (32,)


In [11]:
labels

<tf.Tensor: shape=(32,), dtype=uint8, numpy=
array([3, 4, 6, 2, 3, 4, 8, 6, 4, 2, 6, 3, 9, 0, 1, 0, 8, 3, 6, 0, 8, 0,
       2, 2, 2, 0, 2, 2, 1, 3, 3, 7], dtype=uint8)>